In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np

plt.rcParams["figure.figsize"] = (10,8)

## Lineare Regression für nicht-lineare Daten?

Was aber tun, wenn die Daten **nicht** durch ein lineares Modell abbildbar sind?

Ein Trick, mit dem Sie die lineare Regression an nicht-lineare Beziehungen zwischen
Variablen anpassen können, besteht darin, die Daten mit sogenannten **Basisfunktionen**
zu transformieren.

Die Idee besteht darin, unser mehrdimensionales lineares Modell zu nehmen

$y=a_0+a_1x_1+a_2x_2+a_3x_3+\dots$

und mittels Basisfunktionen zu transformieren: Dazu lassen wir

$x_i = f_i(x)$ für i=1...k

sein, wobei $f_i$ eine Funktion ist, die unsere Daten transformiert.

Wenn zum Beispiel $f_i(x) = x^i$ ist, wird unser Modell zu einer Polynom-Regression:

$y=a_0+a_1x+a_2x^2+a_3x^3+\dots$

Beachten Sie, dass es sich nach wie vor um ein lineares Modell handelt - die
Linearität bezieht sich auf die Tatsache, dass die Koeffizienten $a_i$ sich niemals
miteinander multiplizieren oder dividieren. Was wir effektiv getan haben, ist,
unsere $x$-Werte in eine höhere Dimension zu projizieren, so
dass eine lineare Anpassung kompliziertere Beziehungen zwischen $x$ und $y$
einpassen kann.


## Polynomische Basisfunktionen

Diese Polynomprojektion ist so nützlich, dass sie mit Hilfe des
`PolynomialFeatures-Transformators` in Scikit-Learn eingebaut ist:

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

x = np.array([2, 3, 4])
poly = PolynomialFeatures(3, include_bias=False)
poly.fit_transform(x[:, None])

Wir sehen hier, dass der Transformator unseren skalaren Wert in ein
Array mit drei Elementen umgewandelt hat, indem er Quadrat und dritte Potenz unseres Wertes
als "Feature" erzeugt hat.

Diese neue, höherdimensionale Input-Darstellung kann dann in eine lineare Regression
gesteckt werden. Der sauberste Weg, dies zu erreichen, ist die Verwendung einer sogenannten `Pipeline`.

Erstellen wir auf diese Weise ein Polynom-Modell 7. Grades:

In [ ]:
from sklearn.pipeline import make_pipeline
poly_model = make_pipeline(PolynomialFeatures(7), LinearRegression())

Mit dieser Transformation können wir das lineare Modell verwenden, um viel
kompliziertere Beziehungen zwischen $x$ und $y$ anzupassen. Hier ist zum Beispiel
eine polynomiale Regression einer verrauschte Sinuswelle:

In [ ]:
rng = np.random.default_rng(1)
x = 10 * rng.random(50)
y = np.sin(x) + 0.1 * rng.standard_normal(50)


poly_model.fit(x[:, np.newaxis], y)

xfit = np.linspace(0, 10, 1000)
yfit = poly_model.predict(xfit[:, np.newaxis])

sns.scatterplot(x, y)
sns.lineplot(xfit, yfit);

## Gaußsche Basisfunktionen

Natürlich sind auch andere Basisfunktionen möglich. Ein nützliches Muster ist zum
Beispiel die Anpassung eines Modells, das nicht eine Summe von Polynom-Basen,
sondern eine Summe von Gaußschen Basen ist.
Eine Gaußsche Basis sieht folgendermaßen aus:

In [ ]:
from scipy.stats import norm

x_all = np.arange(-10, 10, 0.001)
y2 = norm.pdf(x_all,0,1)

fig, ax = plt.subplots(figsize=(9,6))

sns.lineplot(x_all,y2, ax=ax)
ax.set_xlim([-4,4])
ax.set_yticklabels([])
ax.set_title('Gauß- / Normal-Verteilung');

 Diese Gaußschen Basisfunktionen sind nicht in Scikit-Learn eingebaut, aber wir
 können einen benutzerdefinierten Transformator schreiben, der sie erzeugt.

 Hier ermöglichen wir also einen Fit der Daten mithilfe eine Summe von (im Beispiel 20) Gauß-Verteilungen:

In [ ]:
import Gauss

gauss_model = make_pipeline(Gauss.GaussianFeatures(20),
                            LinearRegression())
gauss_model.fit(x[:, np.newaxis], y)
yfit = gauss_model.predict(xfit[:, np.newaxis])

sns.scatterplot(x, y)
sns.lineplot(xfit, yfit);

Wir führen dieses Beispiel hier nur an, um deutlich zu machen, dass polynome
Basisfunktionen nichts Magisches an sich haben: Wenn Sie eine Art Intuition in den
Generierungsprozess Ihrer Daten haben, die Sie glauben lässt, dass die eine oder
andere Basis angemessen sein könnte, können Sie sie ebenfalls verwenden.
